In [86]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

Import the list of stock

In [87]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [88]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2300312160731,
 'week52high': 189.46,
 'week52low': 134.52,
 'week52highSplitAdjustOnly': 187.59,
 'week52lowSplitAdjustOnly': 135.14,
 'week52change': 0.020641103819775727,
 'sharesOutstanding': 16459038136,
 'float': 0,
 'avg10Volume': 105566402,
 'avg30Volume': 91305215,
 'day200MovingAvg': 167.47,
 'day50MovingAvg': 146.54,
 'employees': 153008,
 'ttmEPS': 6.3,
 'ttmDividendRate': 0.914665467906018,
 'dividendYield': 0.006744053981192818,
 'nextDividendDate': '',
 'exDividendDate': '2022-05-04',
 'nextEarningsDate': '2022-07-20',
 'peRatio': 22.351089502467804,
 'beta': 1.3365892188290391,
 'maxChangePercent': 52.51331805457949,
 'year5ChangePercent': 3.047932392999814,
 'year2ChangePercent': 0.5800319596041791,
 'year1ChangePercent': 0.03379942338393336,
 'ytdChangePercent': -0.2419054453845684,
 'month6ChangePercent': -0.2124378084738804,
 'month3ChangePercent': -0.18600580954130128,
 'month1ChangePercent': -0.013007889862881905,
 'day30

In [89]:
data['year1ChangePercent']

0.03379942338393336

In [90]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [91]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    try:
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        for symbol in symbol_string.split(','):
            final_dataframe = final_dataframe.append(
            pd.Series([symbol, 
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
            ], 
            index = my_columns), 
                ignore_index = True)
    except:
        pass
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,117.50,-0.221395,N/A
1,AAL,13.50,-0.438374,N/A
2,AAP,181.53,-0.117605,N/A
3,AAPL,139.95,0.033063,N/A
4,ABBV,150.32,0.305102,N/A
...,...,...,...,...
321,YUM,112.95,-0.028645,N/A
322,ZBH,106.76,-0.375412,N/A
323,ZBRA,303.63,-0.413731,N/A
324,ZION,52.62,0.006496,N/A


## Remove low momemtum stock

In [92]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,211,OXY,56.02,1.016129,N/A
1,171,MRO,23.20,0.892789,N/A
2,42,APA,38.90,0.841547,N/A
3,89,CF,89.05,0.806058,N/A
4,168,MOS,52.17,0.71006,N/A
5,109,COP,97.10,0.709224,N/A
6,154,MCK,325.96,0.652512,N/A
7,240,PXD,238.01,0.61076,N/A
8,319,VAR,180.51,0.57812,N/A
9,169,MPC,91.90,0.564444,N/A


## Number of Shares to Buy

In [93]:
portfolio_size = input('Enter the size of your portfolio:')

portfolio_size = float(portfolio_size)
    
    

Enter the size of your portfolio:1000000


In [94]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

C:\Users\Family PC\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,211,OXY,56.02,1.016129,357
1,171,MRO,23.20,0.892789,862
2,42,APA,38.90,0.841547,514
3,89,CF,89.05,0.806058,224
4,168,MOS,52.17,0.71006,383
5,109,COP,97.10,0.709224,205
6,154,MCK,325.96,0.652512,61
7,240,PXD,238.01,0.61076,84
8,319,VAR,180.51,0.57812,110
9,169,MPC,91.90,0.564444,217


## More Advance solution

In [95]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'N/A'
                ]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                [
                    symbol, 
                    data[symbol]['price'], 
                    'N/A', 
                    data[symbol]['stats']['year1ChangePercent'], 
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ], 
                index = hqm_columns), 
                ignore_index = True
            
            )
            
        except:
            pass
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,N/A
0,A,118.20,N/A,-0.219767,N/A,-0.261555,N/A,-0.178484,N/A,-0.092323,N/A,N/A
1,AAL,13.50,N/A,-0.427708,N/A,-0.292383,N/A,-0.192579,N/A,-0.201624,N/A,N/A
2,AAP,177.47,N/A,-0.116229,N/A,-0.247184,N/A,-0.16444,N/A,-0.048866,N/A,N/A
3,AAPL,137.47,N/A,0.034111,N/A,-0.214145,N/A,-0.177467,N/A,-0.012801,N/A,N/A
4,ABBV,151.50,N/A,0.315694,N/A,0.131016,N/A,-0.096451,N/A,-0.050762,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,114.74,N/A,-0.027914,N/A,-0.156081,N/A,-0.054654,N/A,0.001826,N/A,N/A
497,ZBH,108.71,N/A,-0.379814,N/A,-0.166083,N/A,-0.155034,N/A,-0.131018,N/A,N/A
498,ZBRA,302.56,N/A,-0.426798,N/A,-0.492815,N/A,-0.311877,N/A,-0.093695,N/A,N/A
499,ZION,52.88,N/A,0.006634,N/A,-0.140632,N/A,-0.227802,N/A,0.019893,N/A,N/A


In [96]:
time_periods = [
    'One-Year', 
    'Six-Month', 
    'Three-Month', 
    'One-Month'
]
hqm_dataframe['One-Year Price Return'] = hqm_dataframe['One-Year Price Return'].fillna(0)
hqm_dataframe['Six-Month Price Return'] = hqm_dataframe['Six-Month Price Return'].fillna(0)
hqm_dataframe['Three-Month Price Return'] = hqm_dataframe['Three-Month Price Return'].fillna(0)
hqm_dataframe['One-Month Price Return'] = hqm_dataframe['One-Month Price Return'].fillna(0)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100 

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,N/A
0,A,118.20,N/A,-0.219767,0.277445,-0.261555,0.271457,-0.178484,0.371257,-0.092323,0.251497,N/A
1,AAL,13.50,N/A,-0.427708,0.055888,-0.292383,0.231537,-0.192579,0.321357,-0.201624,0.01996,N/A
2,AAP,177.47,N/A,-0.116229,0.467066,-0.247184,0.317365,-0.164440,0.437126,-0.048866,0.528942,N/A
3,AAPL,137.47,N/A,0.034111,0.736527,-0.214145,0.391218,-0.177467,0.379242,-0.012801,0.736527,N/A
4,ABBV,151.50,N/A,0.315694,0.94012,0.131016,0.9002,-0.096451,0.634731,-0.050762,0.510978,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,114.74,N/A,-0.027914,0.606786,-0.156081,0.50499,-0.054654,0.756487,0.001826,0.824351,N/A
497,ZBH,108.71,N/A,-0.379814,0.093812,-0.166083,0.493014,-0.155034,0.46507,-0.131018,0.08982,N/A
498,ZBRA,302.56,N/A,-0.426798,0.057884,-0.492815,0.021956,-0.311877,0.075848,-0.093695,0.243513,N/A
499,ZION,52.88,N/A,0.006634,0.704591,-0.140632,0.532934,-0.227802,0.203593,0.019893,0.878244,N/A


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,N/A,HQM Score
0,A,118.20,N/A,-0.219767,0.277445,-0.261555,0.271457,-0.178484,0.371257,-0.092323,0.251497,N/A,0.292914
1,AAL,13.50,N/A,-0.427708,0.055888,-0.292383,0.231537,-0.192579,0.321357,-0.201624,0.01996,N/A,0.157186
2,AAP,177.47,N/A,-0.116229,0.467066,-0.247184,0.317365,-0.164440,0.437126,-0.048866,0.528942,N/A,0.437625
3,AAPL,137.47,N/A,0.034111,0.736527,-0.214145,0.391218,-0.177467,0.379242,-0.012801,0.736527,N/A,0.560878
4,ABBV,151.50,N/A,0.315694,0.94012,0.131016,0.9002,-0.096451,0.634731,-0.050762,0.510978,N/A,0.746507
...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,114.74,N/A,-0.027914,0.606786,-0.156081,0.50499,-0.054654,0.756487,0.001826,0.824351,N/A,0.673154
497,ZBH,108.71,N/A,-0.379814,0.093812,-0.166083,0.493014,-0.155034,0.46507,-0.131018,0.08982,N/A,0.285429
498,ZBRA,302.56,N/A,-0.426798,0.057884,-0.492815,0.021956,-0.311877,0.075848,-0.093695,0.243513,N/A,0.099800
499,ZION,52.88,N/A,0.006634,0.704591,-0.140632,0.532934,-0.227802,0.203593,0.019893,0.878244,N/A,0.579840


## HQM Score

In [116]:
from statistics import mean

for row in hqm_dataframe.index:
    monentum_percentiles = []
    for time_period in time_periods:
        monentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(monentum_percentiles)
hqm_dataframe

C:\Users\Family PC\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,N/A,HQM Score
0,LB,81.89,244,2.336917,1.0,0.841968,0.998004,0.216826,0.99002,0.080196,0.97006,N/A,0.989521
1,COG,23.31,858,0.342539,0.944112,0.186888,0.928144,0.267028,0.996008,0.256049,0.998004,N/A,0.966567
2,HRB,35.29,566,0.474792,0.964072,0.474615,0.982036,0.311324,0.998004,0.029195,0.91018,N/A,0.963573
3,VRTX,275.24,72,0.443598,0.956088,0.240403,0.94012,0.081142,0.962076,0.045851,0.946108,N/A,0.951098
4,DLTR,157.83,126,0.566456,0.978044,0.146883,0.916168,0.009433,0.892216,0.232409,0.996008,N/A,0.945609
5,NOC,474.26,42,0.266334,0.934132,0.242230,0.942116,0.042444,0.94012,0.051187,0.956088,N/A,0.943114
6,PSX,91.08,219,0.151041,0.878244,0.367024,0.968064,0.204328,0.988024,0.023419,0.89022,N/A,0.931138
7,XOM,91.56,218,0.540740,0.972056,0.544611,0.99002,0.124519,0.974052,-0.004166,0.778443,N/A,0.928643
8,AZO,2132.05,9,0.456242,0.958084,0.021892,0.806387,0.049495,0.9501,0.151159,0.988024,N/A,0.925649
9,HSY,222.40,89,0.258650,0.93014,0.151778,0.92016,0.033163,0.916168,0.039781,0.934132,N/A,0.925150


In [98]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop=True)

In [99]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

C:\Users\Family PC\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,N/A,HQM Score
0,LB,81.89,244,2.336917,1.0,0.841968,0.998004,0.216826,0.99002,0.080196,0.97006,N/A,0.989521
1,COG,23.31,858,0.342539,0.944112,0.186888,0.928144,0.267028,0.996008,0.256049,0.998004,N/A,0.966567
2,HRB,35.29,566,0.474792,0.964072,0.474615,0.982036,0.311324,0.998004,0.029195,0.91018,N/A,0.963573
3,VRTX,275.24,72,0.443598,0.956088,0.240403,0.94012,0.081142,0.962076,0.045851,0.946108,N/A,0.951098
4,DLTR,157.83,126,0.566456,0.978044,0.146883,0.916168,0.009433,0.892216,0.232409,0.996008,N/A,0.945609
5,NOC,474.26,42,0.266334,0.934132,0.242230,0.942116,0.042444,0.94012,0.051187,0.956088,N/A,0.943114
6,PSX,91.08,219,0.151041,0.878244,0.367024,0.968064,0.204328,0.988024,0.023419,0.89022,N/A,0.931138
7,XOM,91.56,218,0.540740,0.972056,0.544611,0.99002,0.124519,0.974052,-0.004166,0.778443,N/A,0.928643
8,AZO,2132.05,9,0.456242,0.958084,0.021892,0.806387,0.049495,0.9501,0.151159,0.988024,N/A,0.925649
9,HSY,222.40,89,0.258650,0.93014,0.151778,0.92016,0.033163,0.916168,0.039781,0.934132,N/A,0.925150


In [100]:
writer = pd.ExcelWriter('monentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index=False)

In [101]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [113]:
column_formats = {
    'A':['Ticker',string_template], 
    'B':['Price', dollar_template], 
    'C':['Number of Shares to Buy', integer_template], 
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column('f{column} : {column} ', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}', 25, column_formats[column][1])

AttributeError: 'NoneType' object has no attribute 'group'

In [114]:
writer.save()

E:\anaconda3\envs\algo_trading\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
